## Generating fixtures to test the Latitude Estimation Function

### Expected Latitude Output is downloaded directly from https://maps.nrel.gov/pvdaq/ PVDAQ Contributed Sites
### Expected latitude = 39.4856

In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
path = Path.cwd().parent.parent.parent
os.chdir(path)

### Load data table from external source

For today's example, we're loading data from NREL's PVDAQ API, which is a publically available PV generatation data set. 

In [2]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 24.3 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




### Running the DataHandler

In [3]:
dh = DataHandler(data_frame[0])
dh.fix_dst()

In [4]:
dh.run_pipeline(power_col='ac_power', fix_shifts=False, correct_tz=False)

total time: 6.98 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              2.51s
Cleaning                   0.13s
Filtering/Summarizing      4.34s
    Data quality           0.19s
    Clear day detect       0.13s
    Clipping detect        3.40s
    Capacity change detect 0.61s



### Estimate declination

In [5]:
from pvsystemprofiler.utilities.declination_equation import delta_spencer

In [6]:
doy = dh.day_index.dayofyear
daily_meas = dh.filled_data_matrix.shape[0]
delta = delta_spencer(doy, daily_meas)

### Estimate hours daylight

In [7]:
from solardatatools.algorithms import SunriseSunset

In [8]:
ss = SunriseSunset()
ss.run_optimizer(data=dh.filled_data_matrix)

In [9]:
sunrise = ss.sunrise_measurements
sunset = ss.sunset_measurements
sunrise[np.isnan(sunrise)] = 0
sunset[np.isnan(sunset)] = 0

In [10]:
hours_daylight = sunset - sunrise

#### Export Inputs for Estimate Latitude

In [11]:
# Hours daylight
np.savetxt('./tests/fixtures/latitude/hours_daylight.csv', hours_daylight,  delimiter=",")
# Delta
np.savetxt('./tests/fixtures/latitude/delta.csv', delta,  delimiter=",")

### Estimate latitude

In [12]:
from pvsystemprofiler.algorithms.latitude.estimation import estimate_latitude

In [13]:
estimate_latitude(hours_daylight, delta)

38.58601372121755